# 如何用Opensearch开发一个向量搜索引擎

## 引用

docker 安装

https://opensearch.org/docs/latest/opensearch/install/docker/

KNN

https://opensearch.org/docs/latest/search-plugins/knn/approximate-knn/

python客户端

https://opensearch.org/docs/latest/clients/python/

文本向量化模型

https://www.sbert.net/docs/pretrained_models.html


## 什么是opensearch

elasticsearch的开源版本

## 什么是向量搜索

word2vector / image2vector / product2vector / document2vector / ...

向量化函数（文本/图片/等等） = 向量

向量化函数（文本） =》 向量

向量 存储到 opensearch

query：需要找到的相似文本

向量化函数（query） =》 去数据库中寻找 （算法：ANN， Approximate Nearest Neighbour，粗召回，精确排序）

## 向量化函数是什么

In [4]:
!pip install -U sentence-transformers >> /dev/null

In [5]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [8]:
sentences = """扎实推动共同富裕，实实在在造福人民
船到中游更奋楫  笑起来真好看|"紫珠珠"变"钱串串"
习近平签署第一〇一至一〇七号主席令 每日一习话
疫情下的西安  封闭管理生活必需品如何保供？
大山有“梦”：一个高山村的寻“富”路
六中全会精神在基层丨小车窗里的时代之变
商务部：中国消费市场稳中向好的趋势没有改变
【地评线】坚定“三新”定位，促进实现高质量发展
寒潮中最北铁路送暖人
尼日利亚毁百万剂欧洲过期疫苗，尼官员怒斥西方国家
铁路部门：不符合进京条件的人员将劝导其在京外下车
在太空出差会"长个"吗?看"感觉良好"乘组如何在太空测身高
明年全国两会召开时间来了！
中美是否会有第二阶段的谈判？商务部回应
17部门：职业病严重的重点行业职工可获工伤保险保障
北京首案已生效：比特币“挖矿”违背民法典“绿色原则”
代购救命药被指贩毒后，1043名癫痫患儿家长高价求药
赵立坚：我见过倒打一耙的 还从没见过这么倒打一耙的
朴槿惠获韩国政府特别赦免 将于12月31日出狱
朴槿惠回应被韩国政府特赦：向总统文在寅“深表谢意” 
普京称北约东扩不可接受 北约：从未承诺过不扩大
一入青瓦台"深似海" 150秒回顾韩国11位前总统的黯淡结局""".split('\n')

embeddings = model.encode(sentences)

In [9]:
embeddings.shape

(22, 384)

# 用Docker的方式运行opensearch

```bash
docker run -p 9200:9200 -p 9600:9600 -e "discovery.type=single-node" opensearchproject/opensearch:1.2.3
```

In [10]:
!pip install opensearch-py >> /dev/null

In [12]:
from opensearchpy import OpenSearch

In [18]:
client = OpenSearch(
    'https://localhost:9200',
    use_ssl=False,
    verify_certs=False,
    http_auth=('admin', 'admin')
)

/opt/conda/lib/python3.8/site-packages/opensearchpy/connection/http_urllib3.py:215: UserWarning: Connecting to https://localhost:9200 using SSL with verify_certs=False is insecure.
  warnings.warn(


In [14]:
# 设置数据库的schema

In [16]:
index = 'python-test'

index_body = {
  "settings": {
    "index": {
      "knn": True,
      "knn.algo_param.ef_search": 100
    }
  },
  "mappings": {
    "properties": {
        "my_vector": {
          "type": "knn_vector",
          "dimension": 384,
          "method": {
            "name": "hnsw",
            "space_type": "l2",
            "engine": "nmslib",
            "parameters": {
              "ef_construction": 128,
              "m": 24
            }
          }
        }
    }
  }
}

In [19]:
client.indices.create(index=index, body=index_body)

/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'python-test'}

In [21]:
for s, emb in zip(sentences, embeddings):
    client.index(index=index, body={
        'my_vector': emb,
        'text': s
    })

/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarnin

In [25]:
hits = client.search(
    index=index,
    body={
        "_source": ["text"],
      "size": 5,
      "query": {
        "knn": {
          "my_vector": {
            "vector": embeddings[0],
            "k": 3
          }
        }
      }
    }
)

/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [28]:
query = '人民群众'

In [29]:
emb =  model.encode([query])[0]

In [30]:
emb.shape

(384,)

In [35]:
hits = client.search(
    index=index,
    body={
      "_source": ["text"],
      "size": 5,
      "query": {
          'bool': {
              'should': [
                  {
                    "knn": {
                      "my_vector": {
                        "vector": emb,
                        "k": 3
                      }
                    }
                  },
                  {
                      'match': {
                          'text': query  # full text search
                      }
                  }
              ]
          }
      }
    }
)

/opt/conda/lib/python3.8/site-packages/urllib3/connectionpool.py:997: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [36]:
hits

{'took': 10,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6, 'relation': 'eq'},
  'max_score': 4.5201745,
  'hits': [{'_index': 'python-test',
    '_type': '_doc',
    '_id': 'PpgY7X0BnZ-R7O7Mf8BK',
    '_score': 4.5201745,
    '_source': {'text': '扎实推动共同富裕，实实在在造福人民'}},
   {'_index': 'python-test',
    '_type': '_doc',
    '_id': 'R5gY7X0BnZ-R7O7MgMB8',
    '_score': 2.3570635,
    '_source': {'text': '寒潮中最北铁路送暖人'}},
   {'_index': 'python-test',
    '_type': '_doc',
    '_id': 'TpgY7X0BnZ-R7O7MgcAX',
    '_score': 2.1600215,
    '_source': {'text': '北京首案已生效：比特币“挖矿”违背民法典“绿色原则”'}},
   {'_index': 'python-test',
    '_type': '_doc',
    '_id': 'SZgY7X0BnZ-R7O7MgMCy',
    '_score': 1.7612586,
    '_source': {'text': '铁路部门：不符合进京条件的人员将劝导其在京外下车'}},
   {'_index': 'python-test',
    '_type': '_doc',
    '_id': 'RJgY7X0BnZ-R7O7MgMBB',
    '_score': 0.101391755,
    '_source': {'text': '六中全会精神在基层丨小车窗里的时代之变'}}]}}